In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
import os
import sys
import nbformat
from langchain_text_splitters import MarkdownHeaderTextSplitter, RecursiveCharacterTextSplitter
from qdrant_client import QdrantClient
from langchain_qdrant import QdrantVectorStore
from langchain_openai import OpenAIEmbeddings

# 현재 노트북 파일의 상위 폴더(Root)를 경로에 추가
root_path = os.path.abspath(os.path.join('..'))
if root_path not in sys.path:
    sys.path.append(root_path)

from src.utils.config import ConfigDB

from dotenv import load_dotenv
load_dotenv(override=True)

# Force-use local embeddings for now to avoid OpenAI API errors
os.environ['EMBEDDING_PROVIDER'] = os.getenv('EMBEDDING_PROVIDER', 'local')
print('EMBEDDING_PROVIDER in kernel:', os.environ.get('EMBEDDING_PROVIDER'))

# Load OPENAI_API_KEY from intro.md into kernel environment (if present)
# NOTE: intro.md is a documentation file and may contain placeholders. We avoid overwriting
# any existing or valid OPENAI_API_KEY already present in the environment.
intro_path = os.path.abspath(os.path.join('..', 'intro.md'))
key = None
try:
    with open(intro_path, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip().startswith('OPENAI_API_KEY'):
                # Split only on the first '=' to allow '=' in values
                _, val = line.split('=', 1)
                key = val.strip().strip("'\"")
                break

    if key:
        # don't use obvious placeholders
        if key in ('', '<your-openai-api-key>', "'<your-openai-api-key>'"):
            print('OPENAI_API_KEY in intro.md looks like a placeholder; not exporting it to os.environ.')
        else:
            current = os.environ.get('OPENAI_API_KEY')
            if current and current != '' and current != '<your-openai-api-key>':
                print('OPENAI_API_KEY already set in environment (not overwritten).')
            else:
                os.environ['OPENAI_API_KEY'] = key
                print('OPENAI_API_KEY set in kernel from intro.md (length=%d)' % len(key))
    else:
        print('OPENAI_API_KEY not found or empty in intro.md')
except FileNotFoundError:
    print('intro.md not found at expected path')

EMBEDDING_PROVIDER in kernel: local
OPENAI_API_KEY already set in environment (not overwritten).


In [2]:
# If your kernel currently shows the placeholder value, reload .env and force it to overwrite the environment
from dotenv import load_dotenv, find_dotenv
# override=True will replace any existing OPENAI_API_KEY in os.environ with the value from .env
env_path = find_dotenv()
if env_path:
    load_dotenv(env_path, override=True)
    key = os.getenv('OPENAI_API_KEY')
    if key and key != '<your-openai-api-key>':
        print('OPENAI_API_KEY loaded from .env (length=%d). To avoid printing the key, it is masked below:' % len(key))
        print('OPENAI_API_KEY (masked):', key[:6] + '...' + key[-6:])
    else:
        print('OPENAI_API_KEY not found in .env or is a placeholder. Check your .env file.')
else:
    print('No .env file found via find_dotenv()')

OPENAI_API_KEY loaded from .env (length=164). To avoid printing the key, it is masked below:
OPENAI_API_KEY (masked): sk-pro...UlNkEA


In [3]:
# ipynb 파일에서 개별 셀(마크다운 / 코드)을 추출하여 메타데이터와 함께 반환
def parse_ipynb_cells(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        nb = nbformat.read(f, as_version=4)
    
    cells = []
    for idx, cell in enumerate(nb.cells):
        if cell.cell_type == 'markdown':
            text = cell.source
            cells.append({'type': 'markdown', 'content': text, 'cell_index': idx})
        elif cell.cell_type == 'code':
            code = cell.source
            # 코드 셀은 코드 블록으로 감싸서 텍스트로 저장
            code_block = f"```python\n{code}\n```"
            cells.append({'type': 'code', 'content': code_block, 'cell_index': idx})
        # 첨부된 이미지나 outputs가 있으면 추후 처리 가능
    
    return cells

In [4]:
# 각 ipynb 파일을 셀 단위로 추출하여 딕셔너리로 보관
result_cells = {}

directory_path = "../data/raw/lectures/"
for file_name in os.listdir(directory_path):
    if file_name.endswith(".ipynb"):
        file_path = os.path.join(directory_path, file_name)
        cells = parse_ipynb_cells(file_path)
        result_cells[file_name] = cells

# 요약 출력
for fn, cells in result_cells.items():
    print(fn, "->", len(cells), "cells")

result_cells

01_머신러닝개요.ipynb -> 31 cells
02_첫번째 머신러닝 분석 - Iris_분석.ipynb -> 63 cells
03_데이터셋 나누기와 모델검증.ipynb -> 86 cells
04_데이터_전처리.ipynb -> 136 cells
05_평가지표.ipynb -> 148 cells
06_과적합_일반화_그리드서치_파이프라인.ipynb -> 161 cells
07_지도학습_SVM.ipynb -> 34 cells
08_지도학습_최근접이웃.ipynb -> 28 cells
09_결정트리와 랜덤포레스트.ipynb -> 95 cells
10_앙상블_부스팅.ipynb -> 38 cells
11_최적화-경사하강법.ipynb -> 21 cells
12_선형모델_선형회귀.ipynb -> 105 cells
13_선형모델_로지스틱회귀.ipynb -> 30 cells
14 군집_Clustering.ipynb -> 29 cells


{'01_머신러닝개요.ipynb': [{'type': 'markdown',
   'content': '# 인공지능 개요',
   'cell_index': 0},
  {'type': 'markdown',
   'content': '## 인공지능 (AI - Artificial Intelligence) 이란\n\n### 지능이란?\n- 지능: 어떤 문제를 해결하기 위한 지적 활동 능력\n- 인공지능\n     - 기계가 사람의 지능을 모방하게 하는 기술\n     - 규칙기반, 데이터 학습 기반',
   'cell_index': 1},
  {'type': 'markdown',
   'content': '### 정의\n- 다트머스대학 수학과 교수인 존 매카시(John McCarthy)가 "지능이 있는 기계를 만들기 위한 과학과 공학" 이란 논문에서 처음으로 제안(1955년)\n- 인간의 지능(인지, 추론, 학습 등)을 컴퓨터나 시스템 등으로 만든 것 또는, 만들 수 있는 방법론이나 실현 가능성 등을 연구하는 기술 또는 과학\n  \n![image.png](attachment:image.png)  ',
   'cell_index': 2},
  {'type': 'markdown',
   'content': '### AGI (Artificial General Intelligence)\n\n1. **정의**  \n   - 인간처럼 **광범위한 지적 과제를 수행할 수 있는 인공지능**  \n   - 새로운 문제를 마주했을 때도 **사전 학습 없이 유연하게 사고하고, 학습하며, 판단** 가능함  \n   - 언어 이해, 논리적 추론, 계획 수립, 감정 인식 등 **다양한 인지 기능을 통합적으로 수행**  \n\n2. **특징**  \n   - 하나의 시스템이 **여러 작업을 동시에 수행** 가능 (예: 번역, 추론, 요약, 게임 등)  \n   - **환경 변화에 적응**하고 **스스로 학습** 가능  \n   - **자기 인식(self-awareness)**, **창의성**,

In [5]:
# 셀 목록(result_cells)을 받아 chunk 단위 Document 목록을 생성
from langchain_text_splitters import MarkdownHeaderTextSplitter, RecursiveCharacterTextSplitter

# Document 클래스를 안전하게 import
try:
    from langchain.schema import Document
except Exception:
    try:
        from langchain.docstore.document import Document
    except Exception:
        # 마지막 대안(가능성 낮음)
        class Document:
            def __init__(self, page_content, metadata=None):
                self.page_content = page_content
                self.metadata = metadata or {}

# Splitter 설정
header_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=[('#', 'Header 1'), ('##', 'Header 2'), ('###', 'Header 3')]
)
code_splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=50)

# 파일당 문서 생성 함수
def prepare_documents_from_cells(file_name, cells):
    """Create Document objects from parsed notebook cells and enrich metadata.

    Adds:
    - lecture_title: derived from file name
    - heading: first markdown heading in the cell (if present)
    - text_snippet: truncated snippet of the chunk
    """
    docs = []
    lecture_title = os.path.splitext(file_name)[0]

    for cell in cells:
        base_meta = {
            'source_file': file_name,
            'lecture_title': lecture_title,
            'cell_index': cell['cell_index'],
            'cell_type': cell['type']
        }
        content = cell['content']

        # Extract a heading when available (first line starting with '#')
        heading = None
        if cell['type'] == 'markdown':
            lines = [ln for ln in content.splitlines() if ln.strip()]
            for ln in lines:
                if ln.strip().startswith('#'):
                    heading = ln.strip().lstrip('#').strip()
                    break
            parts = header_splitter.split_text(content)
        else:
            parts = code_splitter.split_text(content)

        for i, part in enumerate(parts):
            meta = base_meta.copy()
            meta['chunk_id'] = i
            if heading:
                meta['heading'] = heading
            # safe snippet for quick previews
            meta['text_snippet'] = str(part)[:500]
            # Document 객체로 생성
            docs.append(Document(page_content=part, metadata=meta))
    return docs

# 모든 파일에 대해 Documents 생성
all_documents = []
for fn, cells in result_cells.items():
    docs = prepare_documents_from_cells(fn, cells)
    all_documents.extend(docs)

print('Total documents (chunks):', len(all_documents))
# 예시 출력 (robust하게 출력)
for d in all_documents[:3]:
    print('---')
    print('META:', d.metadata)
    content = getattr(d, 'page_content', None)
    if content is None:
        # fallback
        content = getattr(d, 'content', str(d))
    print('CONTENT:', str(content)[:200].replace('\n',' '))

Total documents (chunks): 1001
---
META: {'source_file': '01_머신러닝개요.ipynb', 'lecture_title': '01_머신러닝개요', 'cell_index': 1, 'cell_type': 'markdown', 'chunk_id': 0, 'heading': '인공지능 (AI - Artificial Intelligence) 이란', 'text_snippet': "page_content='- 지능: 어떤 문제를 해결하기 위한 지적 활동 능력\n- 인공지능\n- 기계가 사람의 지능을 모방하게 하는 기술\n- 규칙기반, 데이터 학습 기반' metadata={'Header 2': '인공지능 (AI - Artificial Intelligence) 이란', 'Header 3': '지능이란?'}"}
CONTENT: page_content='- 지능: 어떤 문제를 해결하기 위한 지적 활동 능력 - 인공지능 - 기계가 사람의 지능을 모방하게 하는 기술 - 규칙기반, 데이터 학습 기반' metadata={'Header 2': '인공지능 (AI - Artificial Intelligence) 이란', 'Header 3': '지능이란?'}
---
META: {'source_file': '01_머신러닝개요.ipynb', 'lecture_title': '01_머신러닝개요', 'cell_index': 2, 'cell_type': 'markdown', 'chunk_id': 0, 'heading': '정의', 'text_snippet': 'page_content=\'- 다트머스대학 수학과 교수인 존 매카시(John McCarthy)가 "지능이 있는 기계를 만들기 위한 과학과 공학" 이란 논문에서 처음으로 제안(1955년)\n- 인간의 지능(인지, 추론, 학습 등)을 컴퓨터나 시스템 등으로 만든 것 또는, 만들 수 있는 방법론이나 실현 가능성 등을 연구하는 기술 또는 과학  \n![image.png](attachment:image.png)\

In [6]:
###
# 기존 load_and_split_data는 header split만 수행했지만
# 지금은 파일 단위 셀 추출 -> 셀 유형에 따라 다른 분할기를 적용하도록 변경했습니다.
###

# (위에서 정의한 header_splitter, code_splitter를 사용)
print('Header splitter and code splitter ready')

Header splitter and code splitter ready


In [8]:
# all_documents 변수를 사용 (위에서 생성됨)
print('Documents count:', len(all_documents))

# 첫 몇 개 확인
for d in all_documents[:5]:
    print('---')
    print(d.metadata)
    # 안전한 문자열 변환
    content = getattr(d, 'page_content', None)
    if content is None:
        content = getattr(d, 'content', None)
    if content is None:
        if hasattr(d, 'metadata') and isinstance(d.metadata, dict):
            content = d.metadata.get('text', str(d))
        else:
            content = str(d)
    print('CONTENT:', str(content)[:200].replace('\n',' '))

# 필요시 재분할 파라미터 조정 가능 (chunk_size, overlap 등)

Documents count: 1001
---
{'source_file': '01_머신러닝개요.ipynb', 'lecture_title': '01_머신러닝개요', 'cell_index': 1, 'cell_type': 'markdown', 'chunk_id': 0, 'heading': '인공지능 (AI - Artificial Intelligence) 이란', 'text_snippet': "page_content='- 지능: 어떤 문제를 해결하기 위한 지적 활동 능력\n- 인공지능\n- 기계가 사람의 지능을 모방하게 하는 기술\n- 규칙기반, 데이터 학습 기반' metadata={'Header 2': '인공지능 (AI - Artificial Intelligence) 이란', 'Header 3': '지능이란?'}"}
CONTENT: page_content='- 지능: 어떤 문제를 해결하기 위한 지적 활동 능력 - 인공지능 - 기계가 사람의 지능을 모방하게 하는 기술 - 규칙기반, 데이터 학습 기반' metadata={'Header 2': '인공지능 (AI - Artificial Intelligence) 이란', 'Header 3': '지능이란?'}
---
{'source_file': '01_머신러닝개요.ipynb', 'lecture_title': '01_머신러닝개요', 'cell_index': 2, 'cell_type': 'markdown', 'chunk_id': 0, 'heading': '정의', 'text_snippet': 'page_content=\'- 다트머스대학 수학과 교수인 존 매카시(John McCarthy)가 "지능이 있는 기계를 만들기 위한 과학과 공학" 이란 논문에서 처음으로 제안(1955년)\n- 인간의 지능(인지, 추론, 학습 등)을 컴퓨터나 시스템 등으로 만든 것 또는, 만들 수 있는 방법론이나 실현 가능성 등을 연구하는 기술 또는 과학  \n![image.png](attachment:image.png)\' metadata={\'Header 

In [11]:
# Safe converter for Document-like objects to plain text
def doc_to_text(d):
    t = getattr(d, 'page_content', None)
    if t is None:
        t = getattr(d, 'content', None)
    if t is None:
        if hasattr(d, 'metadata') and isinstance(d.metadata, dict):
            return d.metadata.get('text', str(d))
        return str(d)
    return str(t)

In [24]:
chunk_size = 200
chunk_overlap = 20

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
)

# Sanitize documents: ensure page_content are plain strings (avoid nested Document objects)
sanitized = []
for d in all_documents:
    text = doc_to_text(d)
    meta = getattr(d, 'metadata', {}).copy()
    sanitized.append(Document(page_content=text, metadata=meta))

split_documents = text_splitter.split_documents(sanitized)
print('Split documents count:', len(split_documents))

# Show a few samples for verification
for header in split_documents[:20]:
    content = getattr(header, 'page_content', '')[:200].replace('\n',' ')
    print(">>>>>", content)
    print(header.metadata)

# Replace `all_documents` with the split result so downstream cells use the updated chunks
all_documents = split_documents

Split documents count: 1556
>>>>> page_content='- 지능: 어떤 문제를 해결하기 위한 지적 활동 능력 - 인공지능 - 기계가 사람의 지능을 모방하게 하는 기술 - 규칙기반, 데이터 학습 기반' metadata={'Header 2': '인공지능 (AI - Artificial Intelligence) 이란', 'Header 3': '지능이란?'}
{'source_file': '01_머신러닝개요.ipynb', 'lecture_title': '01_머신러닝개요', 'cell_index': 1, 'cell_type': 'markdown', 'chunk_id': 0, 'heading': '인공지능 (AI - Artificial Intelligence) 이란', 'text_snippet': "page_content='- 지능: 어떤 문제를 해결하기 위한 지적 활동 능력\n- 인공지능\n- 기계가 사람의 지능을 모방하게 하는 기술\n- 규칙기반, 데이터 학습 기반' metadata={'Header 2': '인공지능 (AI - Artificial Intelligence) 이란', 'Header 3': '지능이란?'}"}
>>>>> page_content='- 다트머스대학 수학과 교수인 존 매카시(John McCarthy)가 "지능이 있는 기계를 만들기 위한 과학과 공학" 이란 논문에서 처음으로 제안(1955년) - 인간의 지능(인지, 추론, 학습 등)을 컴퓨터나 시스템 등으로 만든 것 또는, 만들 수 있는 방법론이나 실현 가능성 등을 연구하는 기술 또는 과학
{'source_file': '01_머신러닝개요.ipynb', 'lecture_title': '01_머신러닝개요', 'cell_index': 2, 'cell_type': 'markdown', 'chunk_id': 0, 'heading': '정의', 'text_snippet': 'page_content=\'- 다트머스대학 수학과 교수인 존 매카시(John McCarthy)가 "지능이 있

In [ ]:
# 컬렉션 생성 (먼저 실행!)
from qdrant_client.models import Distance, VectorParams

# 임베딩 모델에 따라 차원 설정
provider = os.getenv('EMBEDDING_PROVIDER', 'openai').lower()
vector_size = 1536 if provider == 'openai' else 384

# 컬렉션이 없으면 생성
if not client.collection_exists(ConfigDB.COLLECTION_NAME):
    client.create_collection(
        collection_name=ConfigDB.COLLECTION_NAME,
        vectors_config=VectorParams(
            size=vector_size,
            distance=Distance.COSINE
        )
    )
    print(f"✅ 컬렉션 '{ConfigDB.COLLECTION_NAME}' 생성 완료! (차원: {vector_size})")
else:
    print(f"⚠️  컬렉션 '{ConfigDB.COLLECTION_NAME}'가 이미 존재합니다.")

In [12]:
# Vector Store 생성 및 업로드 (임베딩 공급자 선택: OPENAI 또는 LOCAL)

provider = os.getenv('EMBEDDING_PROVIDER', 'openai').lower()
print('Embedding provider:', provider)

client = QdrantClient(host="localhost", port=6333)

if provider == 'openai':
    embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")
else:
    # Local sentence-transformers 사용
    try:
        from sentence_transformers import SentenceTransformer
    except Exception:
        %pip install -q sentence-transformers
        from sentence_transformers import SentenceTransformer

    model = SentenceTransformer('all-MiniLM-L6-v2')

    # LangChain의 Embeddings 인터페이스를 구현하는 래퍼
    try:
        from langchain.embeddings.base import Embeddings
    except Exception:
        # fallback import path
        from langchain.embeddings.base import Embeddings

    class LocalHFEmbeddings(Embeddings):
        def __init__(self, model):
            self.model = model

        def embed_documents(self, texts):
            embs = self.model.encode(texts, show_progress_bar=True)
            return [list(map(float, e)) for e in embs]

        def embed_query(self, text):
            e = self.model.encode([text], show_progress_bar=False)[0]
            return list(map(float, e))

    embedding_model = LocalHFEmbeddings(model)

# Qdrant VectorStore에 업로드
# validate_collection_config=False로 설정해 임베딩 차원 불일치 검사 우회
vector_store = QdrantVectorStore(
    client=client,
    collection_name=ConfigDB.COLLECTION_NAME,
    embedding=embedding_model,
    validate_collection_config=False
)

# 안전: 먼저 소량 업로드(테스트) 후 전체 업로드를 진행하세요.
# 외부에서 TEST_UPLOAD 변수를 설정하면 그 값을 사용합니다 (노트북 셀로 제어 가능)
TEST_UPLOAD = globals().get('TEST_UPLOAD', True)
TEST_COUNT = int(os.getenv('TEST_UPLOAD_COUNT', '200'))

if TEST_UPLOAD:
    to_upload = all_documents[:TEST_COUNT]
    print('Uploading (test) documents:', len(to_upload))
else:
    to_upload = all_documents
    print('Uploading all documents:', len(to_upload))

# Documents에 id가 없는 경우를 대비해 ids를 생성하여 전달
# Document 객체를 바로 전달하면 내부에서 page_content가 올바른 문자열이 아닐 경우 문제가 발생하므로
# 명시적으로 텍스트와 메타데이터 리스트를 준비합니다.
texts = [doc_to_text(d) for d in to_upload]
metadatas = [getattr(d, "metadata", {}) for d in to_upload]
import uuid
ids_list = [str(uuid.uuid4()) for _ in to_upload]
# add_texts를 사용해 문자열 리스트를 전달
ids = vector_store.add_texts(texts, metadatas=metadatas, ids=ids_list)
print('Added vectors:', len(ids))
ids

Embedding provider: local
Uploading (test) documents: 200


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Added vectors: 200


['c356aea0-32b1-4cbc-b058-5df508eaf2e6',
 '1800502b-4574-4b80-9c91-c36814086d0e',
 'de1bae46-577e-4997-8fb3-f3a399879bc3',
 '86b3922d-0222-4058-b208-f32c3f87f51b',
 '73849c6e-4d0d-4e86-bab2-8c03caea6f5b',
 '189f1238-a86c-4ffc-ac5f-770ddbecfc0b',
 'd6ddf90d-2197-4b61-bf4d-dba8ac75cd74',
 '8a5bd4d7-948d-476d-af51-705cbc868ee9',
 'e44a341a-1d9f-498c-bd95-f1d8cba6626b',
 'dc10cc6b-b40d-4fc0-98cb-b0270720a19a',
 'd427a82b-10d3-429d-bdc6-e0231db0a408',
 '9cec73f0-85fb-43ac-9f08-518ea17622d2',
 '8b7e54d3-398f-421e-8dc7-d8550e694aa7',
 '9a0ee339-5d62-49e9-851b-1cf3c749b408',
 '59c0729a-4318-4999-803b-de81a8757cb2',
 'cfbb6d1f-1330-4a29-8d9b-65aa4bdb7e18',
 '8b366a04-04a7-400b-9283-31c31fb50df5',
 '3736ef1e-a90f-4e4e-8a69-480a2f2ec25c',
 '3cfe5ef4-250d-4b6b-85d1-4bab0f03e2ce',
 'f50746e2-09bf-4a07-8804-23e126ac941e',
 'a1cc4e33-5f7b-4a64-812f-fc733f869ff9',
 '47ed803e-f6ac-4688-80cf-67947a48097e',
 '1dc05738-9876-46a4-ba69-fd0d4ff0da5c',
 'f381e23f-078c-4646-b144-1bf4fa80eb37',
 '9d4a192c-9dd9-

In [13]:
# Verify total points in Qdrant collection
try:
    cnt = client.count(collection_name=ConfigDB.COLLECTION_NAME)
    print('Qdrant count:', cnt)
except Exception as e:
    print('Count check failed:', e)

Qdrant count: count=2029


In [14]:
# OpenAI key quick test: try a small embedding (will report success/failure)
try:
    # make sure OPENAI_API_KEY is loaded from .env
    from langchain_openai import OpenAIEmbeddings
    emb = OpenAIEmbeddings(model='text-embedding-3-small')
    q = 'test embedding'
    v = emb.embed_query(q)
    print('OpenAI embedding test successful (len=', len(v), ')')
except Exception as e:
    print('OpenAI embedding test failed:', e)
    print('Falling back to local embeddings. To retry, ensure OPENAI_API_KEY in .env is valid and re-run this cell.')

OpenAI embedding test successful (len= 1536 )


In [15]:
# Create a Qdrant snapshot (backup) and download locally
try:
    snapshot = client.create_snapshot(collection_name=ConfigDB.COLLECTION_NAME)
    snapshot_name = snapshot.name
    print('Created snapshot:', snapshot_name)

    # download snapshot to ./snapshots
    from pathlib import Path
    import requests

    Path('./snapshots').mkdir(parents=True, exist_ok=True)
    download_url = f"http://{ConfigDB.HOST}:{ConfigDB.PORT}/collections/{ConfigDB.COLLECTION_NAME}/snapshots/{snapshot_name}"
    out_path = Path('./snapshots') / snapshot_name
    print('Downloading snapshot from', download_url)
    r = requests.get(download_url)
    r.raise_for_status()
    with open(out_path, 'wb') as f:
        f.write(r.content)
    print('Downloaded to local:', out_path)
except Exception as e:
    print('Snapshot creation/download failed:', e)

Created snapshot: learning_ai-5399612789368399-2026-01-05-07-38-15.snapshot
Downloaded to local: snapshots\learning_ai-5399612789368399-2026-01-05-07-38-15.snapshot


In [16]:
# Retriever 테스트: 대표 쿼리로 검색 정확성 검증
try:
    queries = [
        "과적합이란 무엇인가?",
        "교차검증이란 무엇인가?",
        "SVM이 언제 사용되는가?",
    ]

    for q in queries:
        print('\n=== QUERY:', q)
        docs = vector_store.similarity_search(q, k=5)
        for i, d in enumerate(docs[:5]):
            meta = getattr(d, 'metadata', {})
            snippet = getattr(d, 'page_content', '')[:400]
            print(f"[{i}] meta: {meta.get('source_file')} | heading: {meta.get('heading')} | snippet: {snippet[:200].replace('\n',' ')}")

except Exception as e:
    print('Retriever test failed:', e)


=== QUERY: 과적합이란 무엇인가?
[0] meta: 12_선형모델_선형회귀.ipynb | heading: 시각화 | snippet: page_content='##### 시각화'
[1] meta: 08_지도학습_최근접이웃.ipynb | heading: None | snippet: page_content='> ### 유클리디안 거리(Euclidean_distance) ![image.png](attachment:image.png)   \begin{align} &distance = \sqrt{(a_1 - b_1)^2 + (a_2-b_2)^2}\\ &\text{n차원 벡터간의 거리} = \sqrt{(a_1 - b_1)^2 + (a_2-b_
[2] meta: 02_첫번째 머신러닝 분석 - Iris_분석.ipynb | heading: 데이터셋 분할시 주의 | snippet: page_content='#### 데이터셋 분할시 주의 - 분류 문제의 경우 각 클래스(분류대상)가 같은 비율로 나뉘어야 한다.'
[3] meta: 02_첫번째 머신러닝 분석 - Iris_분석.ipynb | heading: 데이터셋 분할시 주의 | snippet: page_content='#### 데이터셋 분할시 주의 - 분류 문제의 경우 각 클래스(분류대상)가 같은 비율로 나뉘어야 한다.'
[4] meta: 02_첫번째 머신러닝 분석 - Iris_분석.ipynb | heading: None | snippet: page_content='#### 데이터셋 분할시 주의 - 분류 문제의 경우 각 클래스(분류대상)가 같은 비율로 나뉘어야 한다.'

=== QUERY: 교차검증이란 무엇인가?
[0] meta: 08_지도학습_최근접이웃.ipynb | heading: None | snippet: page_content='> ### 유클리디안 거리(Euclidean_distance) ![image.png](attachment:image.png)   \begin{align} &distance = \sqrt{(

In [17]:
# Retriever 평가: 정량 지표(precision@k, MRR) 계산 템플릿
from collections import defaultdict

# 사용자가 직접 정답(관련 문서)을 제공해 평가 세트를 구성합니다.
# 예시 형식: {'query': '과적합이란 무엇인가?', 'relevant_files': ['02_첫번째 머신러닝 분석 - Iris_분석.ipynb']}
eval_set = [
    {'query': '과적합이란 무엇인가?', 'relevant_files': ['02_첫번째 머신러닝 분석 - Iris_분석.ipynb', '12_선형모델_선형회귀.ipynb']},
    {'query': '교차검증이란 무엇인가?', 'relevant_files': ['02_첫번째 머신러닝 분석 - Iris_분석.ipynb']},
    {'query': 'SVM이 언제 사용되는가?', 'relevant_files': ['07_지도학습_SVM.ipynb']},
]

k = 5

results = []
for item in eval_set:
    q = item['query']
    gold = set(item['relevant_files'])
    docs = vector_store.similarity_search(q, k=k)
    retrieved_files = [d.metadata.get('source_file') for d in docs]

    # precision@k
    hits = sum(1 for f in retrieved_files if f in gold)
    precision = hits / k

    # MRR
    rr = 0.0
    for rank, f in enumerate(retrieved_files, start=1):
        if f in gold:
            rr = 1.0 / rank
            break

    results.append({'query': q, 'precision@k': precision, 'mrr': rr, 'retrieved': retrieved_files})

# 요약
from statistics import mean
print('Eval results:')
for r in results:
    print(r)

print('\nAverage precision@k:', mean([r['precision@k'] for r in results]))
print('Average MRR:', mean([r['mrr'] for r in results]))

Eval results:
{'query': '과적합이란 무엇인가?', 'precision@k': 0.8, 'mrr': 1.0, 'retrieved': ['12_선형모델_선형회귀.ipynb', '08_지도학습_최근접이웃.ipynb', '02_첫번째 머신러닝 분석 - Iris_분석.ipynb', '02_첫번째 머신러닝 분석 - Iris_분석.ipynb', '02_첫번째 머신러닝 분석 - Iris_분석.ipynb']}
{'query': '교차검증이란 무엇인가?', 'precision@k': 0.8, 'mrr': 0.5, 'retrieved': ['08_지도학습_최근접이웃.ipynb', '02_첫번째 머신러닝 분석 - Iris_분석.ipynb', '02_첫번째 머신러닝 분석 - Iris_분석.ipynb', '02_첫번째 머신러닝 분석 - Iris_분석.ipynb', '02_첫번째 머신러닝 분석 - Iris_분석.ipynb']}
{'query': 'SVM이 언제 사용되는가?', 'precision@k': 1.0, 'mrr': 1.0, 'retrieved': ['07_지도학습_SVM.ipynb', '07_지도학습_SVM.ipynb', '07_지도학습_SVM.ipynb', '07_지도학습_SVM.ipynb', '07_지도학습_SVM.ipynb']}

Average precision@k: 0.8666666666666667
Average MRR: 0.8333333333333334


In [18]:
# Switch to full upload
TEST_UPLOAD = False
print('TEST_UPLOAD set to', TEST_UPLOAD)
print('Documents to upload:', len(all_documents))

TEST_UPLOAD set to False
Documents to upload: 1001


In [19]:
# Check current TEST_UPLOAD value in kernel
try:
    print('Kernel TEST_UPLOAD value:', TEST_UPLOAD)
except NameError:
    print('TEST_UPLOAD not defined')

Kernel TEST_UPLOAD value: False


In [20]:
# 수동 임베딩 및 Qdrant 업서트 (Local 모델 사용, 테스트 배치)
import uuid
from qdrant_client.models import PointStruct

TEST_COUNT = int(os.getenv('TEST_UPLOAD_COUNT', '200'))
to_upload = all_documents[:TEST_COUNT]

# 텍스트 추출 (안전하게 문자열로 변환)
def doc_to_text(d):
    # LangChain Document
    t = getattr(d, 'page_content', None)
    if t is None:
        t = getattr(d, 'content', None)
    if t is None:
        # fallback to metadata text fields
        if hasattr(d, 'metadata') and isinstance(d.metadata, dict):
            return d.metadata.get('text', str(d))
        return str(d)
    return str(t)

texts = [doc_to_text(d) for d in to_upload]

# 모델 로드(이미 로드되어 있지 않으면 로드)
try:
    model
except NameError:
    from sentence_transformers import SentenceTransformer
    model = SentenceTransformer('all-MiniLM-L6-v2')

# 임베딩 계산
embs = model.encode(texts, show_progress_bar=True)

# Point 생성
points = []
for doc, emb in zip(to_upload, embs):
    pid = str(uuid.uuid4())
    payload = doc.metadata.copy()
    payload['text_snippet'] = doc_to_text(doc)[:1000]
    points.append(PointStruct(id=pid, vector=emb.tolist(), payload=payload))

# 업서트
client.upsert(collection_name=ConfigDB.COLLECTION_NAME, points=points)
print('Upserted points:', len(points))

Batches:   0%|          | 0/7 [00:00<?, ?it/s]

Upserted points: 200
